# Sonficiation Project

This worksheet should get you started on the sonification project!

## Packing a sound file

This section will teach you how to make simple sound files with code. First, let's import the necessary function, which is called pack, from the struct library.

In [1]:
from struct import pack
import math

Don't worry too much about what goes into or comes out of the pack function. It essentially allows us to reformat data as we please. Let's use it to make a sound file (we'll be using the .au extension for our sound files) consisting of a single note, which we'll input as a frequency (in cycles/second, or Hertz = Hz).

In [2]:
def make_single_freq(name='test.au', freq=440, dur=1000, vol=0.5):
    """
    creates an AU format file of a sine wave
    of frequency freq (Hz)
    for duration dur (milliseconds)
    at volume vol (max is 1.0)
    """

    #Open the file to write to
    fout = open(name,'wb') #w means write, b means binary mode
    
    #The following line is necessary, but don't worry too much about it. 
    #It creates a "header" for the sound file
    fout.write('.snd'+ pack('>5L',24,8*dur,2,8000,1)) 
    #size, encoding=2, sampling rate = 8000, channel = 1 (above)
    
    
    factor = 2*math.pi*freq/8000 #how you put frequency inside a sine wave
    
    
    #now use a loop to write the data to the sound file
    for t in range(8*dur): 
        #8*dur is the number of "segments" of time in the duration given. 
        #We write the sound to the file in these "segments". 
        #Think of "segment" as a unit of time
        
        
        #sine wave calculations
        sin_segment = math.sin(t*factor)
        fout.write(pack('b',int(vol*127*sin_segment))) #the 127 will be explained later
    
    #Always clean up
    fout.close
        

In [3]:
make_single_freq()

Next, let's make a similar function that instead of a single frequency, takes in a list of frequencies, and writes the sound file containing those frequencies, for the duration given.

Hints:  
1)Divide the duration by the number of frequencies to find out how long each frequency should be sounded.  
2)Don't forget to add the header line.  
3)The pack function expects an integer less than 128. This may cause some trouble. Think about how you might get around that.


In [4]:
def make_multiple_freq(name, freq_list, dur, vol):
    #Open the file and append header line.
    fout = open(name,'wb')
    fout.write('.snd'+ pack('>5L',24,8*dur,2,8000,1))
    
    #calculate how many segments each frequency should last
    seg_per_freq = math.floor(8*dur/len(freq_list))
    
    #since we rounded down, there are some segments not accounted for. How many?
    extra = 8*dur - seg_per_freq*len(freq_list)
    
    #calculate the frequency factor just like in the last function
    freq_factors = []
    for freq in freq_list:
        freq_factors.append(2*math.pi*freq/8000)
    
    break_length=2
    
    #write a loop that makes a "stream" (a list) of numbers (sine segments) to append to the sound file
    t=1
    stream = []
    for factor in freq_factors:
        i=0
        while i<seg_per_freq-break_length:
            sin_segment = math.sin(t*factor)
            stream.append(sin_segment)
            i = i+1
            t=t+1
        j=0
        while j<break_length:
            stream.append(0.0)
            j = j+1
            
            
    #now write the extra segments to the stream, handle this however you want
    i=0
    while i < extra:
        stream.append(0.0)
        i = i+1
    
    #now scale everything in stream to be between -1 and 1.
    max_value = max(abs(min(stream)),abs(max(stream)))
    for k in range(len(stream)):
        stream[k] = stream[k]/max_value
        
    #finally, write everything in the stream to the file
    for k in stream:
        fout.write(pack('b',int(vol*127*k)))
    
    #Always clean up
    fout.close()

In [5]:
scale=[329.63,293.66,261.63,293.66,329.63,329.63,329.63,293.66,293.66,293.66,329.63,392.0,392.0,329.63,293.66,261.63,293.66,329.63,329.63,329.63,329.63,293.66,293.66,329.63,293.66,261.63]

make_multiple_freq('test.au',scale,len(scale)*500,0.5)

Once you have your multiple-frequency function working, try it out by making a sound file that has these frequencies in this order:

261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25

As you saw during the presentations (probably yesterday if you're reading this), this code can be used to write 2-dimensional data. The key to doing this is our ability to write multiple frequencies to a sound file at the same time!

The sound that we hear is a wave. This wave can be modeled with the sin(a\*t) function, where a depends on the frequency of the wave that we want to hear. You've seen this in the functions we wrote above. If we take sine functions with a bunch of different values for a (corresponding to different frequencies), and add them together, it's like we're hitting a bunch of different notes at the same time. This expands our capabilities when generating these sound files.

First we need to decide on a range of frequencies that we want to use. You can experiment with this, but let's use the range of 1000 Hz to 3000 Hz.

Now imagine we have a 2-D array (list) of data (also called a matrix). A small example might look like this:

$$
\begin{bmatrix}
1& 2& 8&\\
4& 1& 2&\\
1& 3& 4&\\
\end{bmatrix}
$$

Let's take a walk through time. We'll start at the top of the matrix, and that will be the beginning of our sound file. Each columnn will correspond to one frequency in the range that we chose. In this simple example, the first column will correspond to the frequency of 1000 Hz, the second column to 2000 Hz, and the 3rd column to 3000 Hz. For the first part of our sound file, the 1000 Hz frequency wave will be multiplied by a 1, the 2000 Hz wave by a 2, and the 3000 Hz wave by an 8 (making the last one the loudest). For the next piece of the sound file, we'll use the 2nd row, and for the 3rd piece, we'll use the 3rd row.

Let's write a function that takes a matrix like this and writes a sound file. You may want to base this code off the multiple-frequency function from earlier (though it may need some changing to account for the sum of multiple waves). Using numpy arrays might be the way to go. Make sure your function can handle a matrix of any size.

More hints:  
1)Use numpy.linspace to divide the columns of the matrix into frequency "bins"
2)You might want to find a way to control the volume. One way to do this is to scale all of the numbers in the matrix to be between 0 and 1.

In [6]:
import numpy

def sonify(name, matrix, dur, vol):
    
    #assign minimum and maximum frequencies
    #freq_min = 200
    freq_min = 261.63
    freq_max = 523.25
    #freq_min = 1000.0
    #freq_max = 3000.0
    
    #Open the file and append header line.
    fout = open(name,'wb')
    fout.write('.snd'+ pack('>5L',24,8*dur,2,8000,1))
    
    #Change 2-D list to a 2-D numpy array for convenience
    data = numpy.asarray(matrix)
    
    #find the maximum value in the array (use numpy.amax)
    max_value=numpy.amax(data)
    
    #find the number of rows and columns of the matrix (use numpy.shape)
    rows,cols = data.shape
    
    #scale the matrix so each value is between 0 and 1
    data = (1.0/max_value)*data
    
    #The number of columns is the number of "frequency bins" we want. Use numpy.linspace to find those frequencies
    freq_list = numpy.linspace(freq_min,freq_max,cols)
    
    #calculate how many segments (in time) each row should last
    seg_per_row = math.floor(8*dur/rows)
    
    #since we rounded down, there are some segments not accounted for. How many?
    extra = 8*dur - seg_per_row*rows
    
    #calculate the frequency factors just like in the last function
    freq_factors = []
    for freq in freq_list:
        freq_factors.append(2*math.pi*freq/8000)
    
    break_length=2
    
    #write a loop (I used a triple loop) to write the stream file like last time
    stream = []
    t=1
    temp = range(cols)
    for m in range(rows):
        i=0
        while i<seg_per_row-break_length:
            value=0
            for n in temp:
                value += data[m,n]*math.sin(t*freq_factors[n])
            stream.append(value)
            i = i+1
            t = t+1
        j=0
        while j<break_length:
            stream.append(0.0)
            j = j+1
      
    #now append the extra segments, handle this however you want
    i=0
    while i < extra:
        stream.append(0)
        i = i+1
        
    #scale just like last time
    max_value = max(abs(min(stream)),abs(max(stream)))
    for k in range(len(stream)):
        stream[k] = stream[k]/max_value
        
    #finally, write everything in the stream to the file
    for k in stream:
        fout.write(pack('b',int(vol*127*k)))
    
    #Always clean up
    fout.close()

In [7]:
myarray = [[1,0,1,0,1,0,0,0],[0,1,0,1,0,1,0,0],[0,0,1,0,1,0,1,0]]

sonify('test.au',myarray,6000,0.5)

Now that you have that function working, let's make a slightly different modification. What we've been doing up until now is called *Amplitude Modulation*. Most applications use something called *Frequency Modulation* instead. The change necessary for this is simple. If x is your matrix element, instead of writing your sine function as x\*sin(factor\*t), move the x inside the sine function. So it's now sin(x\*factor\*t). The effect this has is that now, instead of x controlling the volume of a particular frequency, it is "distorting" that frequency a little.

Change your function to do this.

In [14]:
def sonify(name, matrix, dur, vol):
    
    #assign minimum and maximum frequencies
    freq_min = 200
    #freq_min = 261.63
    #freq_max = 523.25
    #freq_min = 1000.0
    freq_max = 3000.0
    
    #Open the file and append header line.
    fout = open(name,'wb')
    fout.write('.snd'+ pack('>5L',24,8*dur,2,8000,1))
    
    #Change 2-D list to a 2-D numpy array for convenience
    data = numpy.asarray(matrix)
    
    data = numpy.transpose(data)
    
    #find the maximum value in the array (use numpy.amax)
    max_value=numpy.amax(data)
    
    #find the number of rows and columns of the matrix (use numpy.shape)
    rows,cols = data.shape
    
    #scale the matrix so each value is between 0 and 1
    data = (1.0/max_value)*data
    
    #The number of columns is the number of "frequency bins" we want. Use numpy.linspace to find those frequencies
    freq_list = numpy.linspace(freq_min,freq_max,cols)
    
    #calculate how many segments (in time) each row should last
    seg_per_row = math.floor(8*dur/rows)
    
    #since we rounded down, there are some segments not accounted for. How many?
    extra = 8*dur - seg_per_row*rows
    
    #calculate the frequency factors just like in the last function
    freq_factors = []
    for freq in freq_list:
        freq_factors.append(2*math.pi*freq/8000)
    
    break_length=2
    
    #write a loop (I used a triple loop) to write the stream file like last time
    stream = []
    t=1
    temp = range(cols)
    for m in range(rows):
        i=0
        while i<seg_per_row-break_length:
            value=0
            for n in temp:
                value += math.sin(data[m,n]*t*freq_factors[n])
            stream.append(value)
            i = i+1
            t = t+1
        j=0
        while j<break_length:
            stream.append(0.0)
            j = j+1
      
    #now append the extra segments, handle this however you want
    i=0
    while i < extra:
        stream.append(0)
        i = i+1
        
    #scale just like last time
    max_value = max(abs(min(stream)),abs(max(stream)))
    for k in range(len(stream)):
        stream[k] = stream[k]/max_value
        
    #finally, write everything in the stream to the file
    for k in stream:
        fout.write(pack('b',int(vol*127*k)))
    
    #Always clean up
    fout.close()

We've seen before that images are simply 2D-arrays. Find an image, convert it to an array, and find out what kind of sound file it makes. Have fun with it!

In [16]:
import Image

myarray = numpy.asarray(Image.open('qr.jpg').convert('L'))

sonify('test.au',myarray,8000,0.8)

Now we're ready to look at data! Can you think of any other instances where data might be stored in a 2-dimensional array? Think for a little, but if you can't come up with something, ask one of your TAs. Find some different instances of such data and sonify them.